In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [2]:
# fix the contractions


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

import re
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


train['text'] = train['text'].apply(replace_contractions)
test['text'] = test['text'].apply(replace_contractions)

train.text = train.text.apply(lambda x : decontracted(x))
test.text = test.text.apply(lambda x : decontracted(x))


We will be using the pretrained BertForSequenceClassifier network. For that reason we do minimal prepocessing here, only getting rid of weird characters and @ and # while keeping the words. We also keep the url's as number of urls, whehter there is a link or not might be an indicator.

In [3]:
train.text = train.text.apply(lambda x: re.sub("[^a-zA-Z.!?0-9]", " ",x))
test.text = test.text.apply(lambda x: re.sub("[^a-zA-Z.!?0-9]", " ",x))

In [4]:
!pip install transformers
features = train.text
labels = train.target

     |████████████████████████████████| 1.4MB 8.9MB/s 
     |████████████████████████████████| 890kB 24.0MB/s 
     |████████████████████████████████| 2.9MB 42.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0616caecae4cc4d9c41622b59885cd677144b5f20544f005a30eda7443039974
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [6]:
# from pytorch_pretrained_bert import BertTokenizer, BertConfig
from transformers import  BertTokenizer, BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)
# For every sentence...
for sent in features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [7]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)


# Calculate the number of samples to include in each set.
train_size = int(0.85 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [9]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-large-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [10]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [11]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [22]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [13]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [14]:
import random
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):

    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch    40  of    203.    Elapsed: 0:00:31.
  Batch    80  of    203.    Elapsed: 0:01:02.
  Batch   120  of    203.    Elapsed: 0:01:33.
  Batch   160  of    203.    Elapsed: 0:02:04.
  Batch   200  of    203.    Elapsed: 0:02:35.

  Average training loss: 0.45
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.858
  Validation took: 0:00:08

======== Epoch 2 / 2 ========
Training...
  Batch    40  of    203.    Elapsed: 0:00:31.
  Batch    80  of    203.    Elapsed: 0:01:01.
  Batch   120  of    203.    Elapsed: 0:01:32.
  Batch   160  of    203.    Elapsed: 0:02:03.
  Batch   200  of    203.    Elapsed: 0:02:34.

  Average training loss: 0.32
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.866
  Validation took: 0:00:08

Training complete!


In [15]:
ids = test['id']
texts = test['text'].values
preds = []
for sent in texts:
    model.eval()

    encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )
        
        # Add the encoded sentence to the list.    
    b_input_ids = encoded_dict['input_ids']
        
        # And its attention mask (simply differentiates padding from non-padding).
    b_input_mask = encoded_dict['attention_mask']
    batch = b_input_ids, b_input_mask

    batch = tuple(t.to(device) for t in batch)
          
        # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.

    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    sentence_ = tokenizer.decode(b_input_ids[0])
    preds.append(np.argmax(logits, axis=1).flatten()[0])


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
df = pd.DataFrame({'id':ids, 'target':preds})
df.to_csv('submission.csv', index=False)

In [17]:
del model

Above, we did some preprocessing and used BertForSequenceClassifier for prediction. 

Next, we investigate if we can improve the model by adding features; such as number of '?', number of '!', number of mentions, number of hashtags, number of emojis, number of digits. To add these features, we need to build a new model which will use BertModel as base and then will connect a classification layer on top. Before feeding the BertModel output to the classificaiton layer, we will concatenate additional features to the output and then use the resulting output to feed into to the classification layer. 

In [3]:
def extract_num_hashtags(s):
    return s.count('#')

def extract_num_urls(s):
    return s.count('http')

def extract_num_mentions(s):
    return s.count('@')

def extract_num_exclamations(s):
    return s.count('!')

def extract_num_qmarks(s):
    return s.count('?')

def extract_dig_count(s):
    dig_count = 0
    for i in range(10):
      dig_count += s.count(str(i))

    return dig_count

def extract_num_emojis(s):
    emoji_counter = 0
    data = re.findall(r'\\X', s)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_counter += 1
            # Remove from the given text the emojis
            s = s.replace(word, '') 
    #words_counter = len(text.split())
    return emoji_counter


train['num_tags'] = train['text'].apply(extract_num_hashtags)
train['num_urls'] = train['text'].apply(extract_num_urls)
train['num_mentions'] = train['text'].apply(extract_num_mentions)
train['num_exclamations'] = train['text'].apply(extract_num_exclamations)
train['num_qmarks'] = train['text'].apply(extract_num_qmarks)
train['num_digits'] = train['text'].apply(extract_dig_count)



test['num_tags'] = test['text'].apply(extract_num_hashtags)
test['num_urls'] = test['text'].apply(extract_num_urls)
test['num_mentions'] = test['text'].apply(extract_num_mentions)
test['num_exclamations'] = test['text'].apply(extract_num_exclamations)
test['num_qmarks'] = test['text'].apply(extract_num_qmarks)
test['num_digits'] = test['text'].apply(extract_dig_count)


In [4]:
#As we have #digits, #urls as additional features, we will remove them from the text.

train.text = train.text.apply(lambda x: re.sub(r"http\S+", "", x))
test.text = test.text.apply(lambda x: re.sub(r"http\S+", "", x))

train.text = train.text.apply(lambda x: re.sub('https?://[A-Za-z0-9./]+','', x))
test.text = test.text.apply(lambda x: re.sub('https?://[A-Za-z0-9./]+','', x))



train.text = train.text.apply(lambda x: re.sub("[^a-zA-Z0-9?!,.@#/:\/]", " ",x))
test.text = test.text.apply(lambda x: re.sub("[^a-zA-Z0-9?!.,@#/:\/]", " ",x))

# train.text = train.text.apply(lambda x: re.sub("[^a-zA-Z?!.;,@#]", " ",x))
# test.text = test.text.apply(lambda x: re.sub("[^a-zA-Z?!.;,@#]", " ",x))

def extract_hash_tags(s):
    return list(set(part[1:] for part in s.split() if part.startswith('#')))

def extract_mentions(s):
    return list(set(part[1:] for part in s.split() if part.startswith('@')))

train['text_tags'] = train['text'].apply(extract_hash_tags)
train['mentions'] = train['text'].apply(extract_mentions)

test['text_tags'] = test['text'].apply(extract_hash_tags)
test['mentions'] = test['text'].apply(extract_mentions)

# train.text = train.text.apply(lambda x : fix_punc(x))
# test.text = test.text.apply(lambda x : fix_punc(x))


In [5]:
import itertools, collections
all_words_no_urls = list(itertools.chain(*train.text_tags))

# Create counter
counts_no_urls = collections.Counter(all_words_no_urls)

counts_no_urls.most_common(50)

[('news', 36),
 ('News', 36),
 ('best', 30),
 ('hot', 30),
 ('prebreak', 30),
 ('Hiroshima', 21),
 ('??', 17),
 ('GBBO', 16),
 ('', 13),
 ('terrorism', 11),
 ('ISIS', 11),
 ('RT', 10),
 ('nowplaying', 10),
 ('???', 10),
 ('India', 10),
 ('BBC', 10),
 ('Sismo', 10),
 ('islam', 10),
 ('Directioners', 9),
 ('NowPlaying', 9),
 ('Japan', 9),
 ('jobs', 9),
 ('CNN', 9),
 ('yyc', 9),
 ('earthquake', 8),
 ('EDM', 8),
 ('DnB', 8),
 ('Beyhive', 8),
 ('world', 8),
 ('abstorm', 8),
 ('Nursing', 8),
 ('Islam', 8),
 ('Dubstep', 7),
 ('Ices', 7),
 ('TrapMusic', 7),
 ('Dance', 7),
 ('IranDeal', 7),
 ('Earthquake', 7),
 ('WX', 7),
 ('animalrescue', 7),
 ('god', 7),
 ('Truth', 7),
 ('Quran', 7),
 ('Lies', 7),
 ('US', 6),
 ('MTVHottest', 6),
 ('Handbag', 6),
 ('Womens', 6),
 ('tcot', 6),
 ('Seattle', 6)]

The below block is for when we tested using Named Entity Recognition (NER) to modify the tweets. However, it end up reducing the accuracy. NER identifies if the entities in a text, such as Person, Organization, Location. The idea was that if we replace a specific named location, specific named person and a specific named organization with the words location, person, organization respectively, we might gain performance - since then it'll be easier to recognize for example if someone is talking about a disaster, they are most likely mentioning or hashtaging an organization like a news outlet and location like the location of the disaster. And so one disaster tweet might have 'Houston' whereas the other might have 'New York'. And the specific names are not necessarily correlated with disaster, and so instead we just replace them with location, maybe there will be a relationship between disaster tweet having location mentioned. At least that was the idea.


In [6]:



# # document = 'what a night! Lakers are playing Rockets at Houston tonight, while Yosemite is on fire and the police arresting people, and FEMA hanging with Will'
# # document = 'Steve, Will, what a Monday night! Lakers are playing Rockets at Houston tonight, while Yosemite is on fire and the police arresting people, and FEMA hanging with Will'
# !pip3 install -U git+https://github.com/aboSamoor/polyglot.git@master
# !polyglot download embeddings2.en ner2.en
# from polyglot.text import Text


# updated_text = []
# for i,row in train.iterrows():
#   document = row['text']


#   document_decoy = document.split()
  
#   try:
#     entity = Text(document).entities
  
#     for entity in Text(document).entities:
    
#       tag = entity.tag
#       word = entity
#       if len(word) >= 1:
#         for j in word:
#           try:
#             ind = document_decoy.index(j)
#           except:
#             continue
#           if tag == 'I-PER':
#             tag = 'person'
#           elif tag == 'I-LOC':
#             tag = 'location'
#           elif tag == 'I-ORG':
#             tag = 'organization'
#           document_decoy[ind] = tag      

    
#     updated = " ".join(document_decoy)
#     updated_text.append(updated)
  
#   except:
#     updated_text.append(document)


  

# train['text'] = updated_text

# updated_text = []
# for i,row in test.iterrows():
#   document = row['text']


#   document_decoy = document.split()
  
#   try:
#     entity = Text(document).entities
  
#     for entity in Text(document).entities:
    
#       tag = entity.tag
#       word = entity
#       if len(word) >= 1:
#         for j in word:
#           try:
#             ind = document_decoy.index(j)
#           except:
#             continue
#           if tag == 'I-PER':
#             tag = 'person'
#           elif tag == 'I-LOC':
#             tag = 'location'
#           elif tag == 'I-ORG':
#             tag = 'organization'
#           document_decoy[ind] = tag      

    
#     updated = " ".join(document_decoy)
#     updated_text.append(updated)
  
#   except:
#     updated_text.append(document)


  

# test['text'] = updated_text

In [7]:
train.text = train.text.apply(lambda x: re.sub("[^a-zA-Z.!?]", " ",x))
test.text = test.text.apply(lambda x: re.sub("[^a-zA-Z.!?]", " ",x))

In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast, BertModel

# specify GPU
device = torch.device("cuda")

In [9]:
# split train dataset into train, validation and test sets
train_feats = train.copy()
train_feats.drop(['target'], axis=1, inplace=True)

In [10]:
train, val, train_labels, val_labels = train_test_split(train_feats, train['target'], 
                                                                    random_state=42, 
                                                                    test_size=0.15, 
                                                                    stratify=train['target'])

In [11]:
# import BERT-base pretrained model
bert = BertModel.from_pretrained('bert-large-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased')

In [12]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train.text.tolist(),
    add_special_tokens=True,
    max_length = 64,
    pad_to_max_length=True,
    truncation=True,
    return_attention_mask=True,
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val.text.tolist(),
    add_special_tokens=True,
    max_length = 64,
    pad_to_max_length=True,
    truncation=True,
    return_attention_mask=True
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test.text.tolist(),
    add_special_tokens=True,
    max_length = 64,
    pad_to_max_length=True,
    truncation=True,
    return_attention_mask=True
)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
## convert lists to tensors

train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
# test_y = torch.tensor(test_labels.tolist())

In [14]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

torch.manual_seed(42)

#define a batch size
batch_size = 64



extra_feat_tr=torch.tensor(np.column_stack((train.num_tags, train.num_urls, train.num_mentions, train.num_exclamations, train.num_qmarks, train.num_digits)))

extra_feat_val=torch.tensor(np.column_stack((val.num_tags, val.num_urls, val.num_mentions, val.num_exclamations, val.num_qmarks, val.num_digits)))


# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y, extra_feat_tr)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y, extra_feat_val)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [15]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()


      # dense layer 1
      self.fc1 = nn.Linear(1024 + 6, 2)
      # self.fc1 = nn.Linear(1024, 2)
      

    #define the forward pass
    def forward(self, sent_id, mask, extra):

      #pass the inputs to the model  
      outputs = self.bert(sent_id, attention_mask=mask)
      x = self.dropout(outputs[1])
      x = self.fc1(torch.cat((x, extra), dim=1))
      # x = self.fc1(x)

      return x

In [16]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [17]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),
                  lr = 1e-5)          # learning rate

In [18]:
# number of training epochs
epochs = 3


from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [19]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels, extra = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask, extra)


    # compute the loss between actual and predicted values
    # loss = cross_entropy(preds, labels)
    # loss = loss(preds.view(-1, 2), labels.view(-1))

    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(preds.view(-1, 2), labels.view(-1))


    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    scheduler.step()  

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [20]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []


  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = time.time() - t0
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels, extra = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      logits = model(sent_id, mask, extra)

      # compute the validation loss between actual and predicted values
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, 2), labels.view(-1))

      total_loss = total_loss + loss.item()

      label_ids = labels.to('cpu').numpy()

      preds = logits.detach().cpu().numpy()
      total_preds.append(preds)

      tmp_eval_accuracy = flat_accuracy(preds, label_ids)
            
      # Accumulate the total accuracy.
      eval_accuracy += tmp_eval_accuracy
      # Track the number of batches
      nb_eval_steps += 1
  # Report the final accuracy for this validation run.
  print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))


  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds, eval_accuracy/nb_eval_steps

In [ ]:
# set initial loss to infinite
best_valid_acc = 0.0

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _, valid_acc= evaluate()
    
    #save the best model
    if valid_acc >= best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')
    print(f'Validation Acc: {valid_acc:.3f}')


In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

In [ ]:
ids = test['id']
texts = test['text'].values
preds = []
for i, row in test.iterrows():
    model.eval()

    sent = row['text']
    tokens_test = tokenizer.batch_encode_plus(
    [sent],
    add_special_tokens=True,
    max_length = 64,
    pad_to_max_length=True,
    truncation=True,
    return_attention_mask=True
    )

    test_seq = torch.tensor(tokens_test['input_ids'])
    test_mask = torch.tensor(tokens_test['attention_mask'])

    extra_feat_test=torch.tensor(np.column_stack((row.num_tags, row.num_urls, row.num_mentions, row.num_exclamations, row.num_qmarks, row.num_digits)))

        
    batch = test_seq, test_mask, extra_feat_test


    batch = [t.to(device) for t in batch]

    sent_id, mask, extra = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      logits = model(sent_id, mask, extra)

      # compute the validation loss between actual and predicted values

      preds.append(logits.detach().cpu().numpy()[0])



In [ ]:
preds = np.argmax(preds, axis=1).flatten()

In [ ]:
df = pd.DataFrame({'id':ids, 'target':preds})
df.to_csv('submission.csv', index=False)

In [ ]:
# NOTE: training loop and transfer learning code was taken from : https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
# and then modified to our needs